In [130]:
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

import pyperclip
import time

In [131]:
def selenium_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--proxy-server=socks5://127.0.0.1:9150")
    options.add_argument('window-size=1920x1080')
    options.add_argument('disable-gpu')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.add_argument('--no-sandbox')
    options.add_argument('no-sandox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--start-maximized')
    options.add_argument('incognito')
    
    service = Service(ChromeDriverManager().install())
    browser = webdriver.Chrome(service=service, options=options)
    
    return browser

In [132]:
def top_to_bottom(browser):
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(1.5)

        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height  

In [133]:
def search(browser, i_keyword, i_city = None):
    try:
        #open browser
        browser.get("https://www.facebook.com/")
#         time.sleep(2)
        
        #log info, search keyword
        USER = "shm8485@gmail.com"
        PWD = "sohee8485"
        KEYWORD = i_keyword
        CITY = i_city
        
        #login
        elem_id = browser.find_element("id", "email")
        pyperclip.copy(USER)
        elem_id.send_keys(Keys.CONTROL, "v")
        time.sleep(2)
        
        elem_pw = browser.find_element("id", "pass")
        pyperclip.copy(PWD)
        elem_pw.send_keys(Keys.CONTROL, "v")
        elem_pw.send_keys("\n")
        time.sleep(3)
        
        #search
        browser.get(f"https://www.facebook.com/search/people/?q={KEYWORD}")
        time.sleep(2)
        
        
        try:
            elem_block = browser.find_element("xpath", "/html/body/div[3]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div")
            elem_block.click()
        except:
            pass
        try:
            elem_block = browser.find_element('xpath', '/html/body/div[5]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div')
            elem_block.click()
        except:
            pass
        time.sleep(1)
        
        #city
        if CITY:
            elem_city = browser.find_element("xpath", "/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div/div/div/div/div")
            elem_city.click()
            elem_city = browser.find_element("xpath", '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[3]/div[2]/div[2]/div/div/div/div/div/div/input')
            time.sleep(1)
            pyperclip.copy(CITY)
            elem_city.send_keys(Keys.CONTROL, "v")
            time.sleep(1)         
            elem_city = browser.find_element('xpath', '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[2]/div/div/div[1]/div[1]/div/ul/li[1]/div/div[1]/div/div/div/div/div[2]/div/span/span')
            elem_city.click()
            time.sleep(1)      
        
    except:
        browser.save_screenshot("screenshot_.png")

In [134]:
def get_links(browser):
    soup_profile = bs(browser.page_source, "html.parser")
    top_elements = soup_profile.find_all(class_="x1yztbdb")
    final_href = []

    for i in top_elements:
        final_href.append(i.find("a").attrs['href'])
    
    final_list = []
    url_id = 'php?id='

    for i in final_href:
        if url_id in i:
            href = i[:len(i)-11]+'&sk='
        else:
            href = i[:len(i)-11]+'/'
        final_list.append(href)
    return final_list

In [135]:
def print_csv(f_name, name, contact, city):
        df = pd.DataFrame({"이름":name,
                           "연락처":contact,
                           "지역":city})
        df.to_csv(f"{f_name}.csv", encoding='UTF-8')

In [136]:
def crawling(f_name, keyword, city= None):
    browser = selenium_driver()
    search(browser, keyword+'\n', city)
    
    #scroll to bottom
    top_to_bottom(browser)
        
    #scrap the profiles' link
    final_list = get_links(browser)
    name, contact, city = [], [], []
    
    for i in final_list:
        try:
            browser.get(i+"about_contact_and_basic_info")
            time.sleep(1)
            soup_info = bs(browser.page_source, "html.parser")
            soup_name = soup_info.find(class_="x78zum5 x15sbx0n x5oxk1f x1jxijyj xym1h4x xuy2c7u x1ltux0g xc9uqle")
            name.append(soup_name.find("h1").text)
            soup_info_contact = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
            contact.append(soup_info_contact.text)
            browser.get(i+"about_places")
            time.sleep(1)
            soup_info = bs(browser.page_source, "html.parser")
            soup_info_ct = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
            city.append(soup_info_ct.text)
        except:
            l_name = len(name)
            l_cont = len(contact)
            l_city = len(city)
            if l_name > l_cont:
                contact.append('empty')
                print(1)
                try:
                    browser.get(i+"about_places")
                    time.sleep(1)
                    soup_info = bs(browser.page_source, "html.parser")
                    soup_info_ct = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
                    city.append(soup_info_ct.text)
                except:
                    city.append('')
            else:
                city.append('')
                
    print_csv(f_name, name, contact, city)


In [137]:
def main(f_name, keyword, city=None):
    start = time.time()
    #crawling(file_name, keyword, city)
    crawling(f_name, keyword, city)
    print("소요 시간: ", time.time() - start)

In [140]:
if __name__ == "__main__":
    main('test_tracvel''여행')

소요 시간:  944.2355122566223
